In [4]:
#Written by Roc Chow, March 2020

In [5]:
#import libraries
import pandas as pd
import numpy as np
import glob
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# read all csv files under the same directory
def readallcsv():
    allcsv = glob.glob("./*.csv")
    print("The following DataFrame will be created: ")
    for f in allcsv:
        print(f.split('.')[1][1:])
        globals()[f.split('.')[1][1:]] = pd.read_csv(f)


In [7]:
readallcsv()

The following DataFrame will be created: 
calendar
sales_train_validation
sample_submission
sell_prices


In [8]:
# Preparing Dataframe

# adding states and item category column to sell_prices df
sell_prices['state'] = sell_prices.store_id.str.split("_", n = 1, expand = True)[0]
sell_prices['category'] = sell_prices.item_id.str.split("_", n = 1, expand = True)[0]

#joining calendar and sales
joined_df = pd.merge(sell_prices, calendar, on='wm_yr_wk')

In [9]:
joined_df.head()

,store_id,item_id,wm_yr_wk,sell_price,state,category,date,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,CA_1,HOBBIES_1_001,11325,9.58,CA,HOBBIES,2013-07-13,Saturday,1,7,2013,d_897,NaN,NaN,NaN,NaN,0,1,0
1,CA_1,HOBBIES_1_001,11325,9.58,CA,HOBBIES,2013-07-14,Sunday,2,7,2013,d_898,NaN,NaN,NaN,NaN,0,0,1
2,CA_1,HOBBIES_1_001,11325,9.58,CA,HOBBIES,2013-07-15,Monday,3,7,2013,d_899,NaN,NaN,NaN,NaN,0,1,1
3,CA_1,HOBBIES_1_001,11325,9.58,CA,HOBBIES,2013-07-16,Tuesday,4,7,2013,d_900,NaN,NaN,NaN,NaN,0,0,0
4,CA_1,HOBBIES_1_001,11325,9.58,CA,HOBBIES,2013-07-17,Wednesday,5,7,2013,d_901,NaN,NaN,NaN,NaN,0,0,0


In [10]:
# correlation matrix
corr_matrix = joined_df.corr()
corr_matrix["sell_price"].sort_values(ascending=False)

sell_price    1.000000
wm_yr_wk      0.019481
year          0.019100
month         0.000503
snap_WI       0.000034
snap_TX       0.000029
wday         -0.000033
snap_CA      -0.000084
Name: sell_price, dtype: float64

In [11]:
# min df, aim to try with these min features to train, more features to add later to improve
min_df = joined_df[['store_id', 'item_id', 'wm_yr_wk', 'sell_price', 'state', 'category','date', 'weekday', 'wday', 'month', 'year', 'd']]

In [12]:
min_df.shape

(47735397, 12)